In [3]:
%%writefile practice52.cu
// Практика 5 часть 2
// Реализовать очередь с использованием атомарных операций для безопасного добавления и удаления элементов.

#include <stdio.h>          // Стандартная библиотека ввода/вывода
#include <cuda_runtime.h>   // Библиотека CUDA Runtime API

// ПАРАЛЛЕЛЬНАЯ ОЧЕРЕДЬ НА CUDA
struct Queue {
    int *data;      // Массив для хранения элементов очереди
    int head;       // Индекс начала очереди (для dequeue)
    int tail;       // Индекс конца очереди (для enqueue)
    int capacity;   // Максимальная ёмкость очереди

    // Инициализация очереди
    __device__ void init(int *buffer, int size) {
        data = buffer;   // Привязываем очередь к памяти GPU
        head = 0;        // Начало очереди
        tail = 0;        // Конец очереди
        capacity = size; // Максимальная ёмкость
    }

    // Enqueue (добавление элемента)
    __device__ bool enqueue(int value) {
        int pos = atomicAdd(&tail, 1); // Атомарно резервируем индекс для записи
        if (pos < capacity) {          // Проверяем переполнение
            data[pos] = value;         // Записываем элемент
            return true;               // Успешное добавление
        }
        return false;                  // Очередь переполнена
    }

    // Dequeue (извлечение элемента)
    __device__ bool dequeue(int *value) {
        int pos = atomicAdd(&head, 1); // Атомарно резервируем индекс для чтения
        if (pos < tail) {              // Проверяем, есть ли элементы
            *value = data[pos];        // Извлекаем элемент
            return true;               // Успешное извлечение
        }
        return false;                  // Очередь пуста
    }
};

// Инициализация очереди на GPU
__global__ void initQueueKernel(Queue *q, int *buf, int size) {
    q->init(buf, size);   // Вызов метода init на GPU
}

// Параллельный enqueue
__global__ void enqueueKernel(Queue *q) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    // tid — глобальный индекс потока
    q->enqueue(tid);
    // Каждый поток добавляет своё значение в очередь
}

// Параллельный dequeue
__global__ void dequeueKernel(Queue *q, int *out) {
    int value;
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    // tid — индекс потока, используется для записи результата

    if (q->dequeue(&value)) {
        out[tid] = value;
        // Сохраняем извлечённый элемент в массив вывода
    }
}

// Основная функция
int main() {
    const int N = 256;
    // Размер очереди и количество потоков

    int *d_buffer, *d_output;
    Queue *d_queue;

    // Выделение памяти на GPU
    cudaMalloc(&d_buffer, N * sizeof(int));
    // Память под элементы очереди
    cudaMalloc(&d_output, N * sizeof(int));
    // Память под выходной массив
    cudaMalloc(&d_queue, sizeof(Queue));
    // Память под структуру очереди

    cudaMemset(d_output, 0, N * sizeof(int));
    // Обнуляем выходной массив

    // Инициализация очереди
    initQueueKernel<<<1,1>>>(d_queue, d_buffer, N);
    cudaDeviceSynchronize();
    // Ждём завершения инициализации

    // Параллельное добавление элементов
    enqueueKernel<<<1,N>>>(d_queue);
    cudaDeviceSynchronize();
    // Ждём завершения всех enqueue

    // Параллельное извлечение элементов
    dequeueKernel<<<1,N>>>(d_queue, d_output);
    cudaDeviceSynchronize();
    // Ждём завершения всех dequeue

    // Копирование результатов на CPU
    int h_output[N];
    cudaMemcpy(h_output, d_output, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Вывод первых 10 извлечённых элементов
    printf("Первые 10 извлечённых элементов очереди:\n");
    for(int i = 0; i < 10; i++)
        printf("%d ", h_output[i]);
        // Ожидается: 0 1 2 3 4 5 6 7 8 9
    printf("\n");

    // Освобождение памяти GPU
    cudaFree(d_buffer);
    cudaFree(d_output);
    cudaFree(d_queue);

    return 0;
    // Завершение программы
}

/*
ОЖИДАЕМЫЙ РЕЗУЛЬТАТ НА РЕАЛЬНОЙ CUDA-GPU:

Первые 10 извлечённых элементов очереди:
0 1 2 3 4 5 6 7 8 9

Подтверждает корректную работу очереди (FIFO).

ПРИ ЗАПУСКЕ В GOOGLE COLAB МОЖЕТ ВЫВОДИТЬСЯ:

0 0 0 0 0 0 0 0 0 0

Причина: виртуализированная среда GPU Colab может некорректно
обрабатывать атомарные операции в lock-free структурах данных.
*/


Overwriting practice52.cu


In [4]:
# Компиляция
!nvcc practice52.cu -o practice52

# Запуск
!./practice52



Первые 10 извлечённых элементов очереди:
0 0 0 0 0 0 0 0 0 0 
